In [9]:
import os
import re
import glob
import difflib
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date
from dateutil import relativedelta
from sklearn import preprocessing, metrics
from sklearn.cluster import AgglomerativeClustering, KMeans, MeanShift, FeatureAgglomeration
%matplotlib inline

# LOAD DATA TO DICTIONARY

In [10]:
path = 'C:/analyticsdev/Projects/EXPECTEDX/Alloe/data'
files = glob.glob(os.path.join(path, "*.csv"))

raw_data = [pd.read_csv(name) for name in files]
df_names = [re.findall('(?<=\.)[a-zA-Z]+(?=\.)', names) for names in files]
df_names = itertools.chain(*df_names)
raw_data = dict(zip(df_names, raw_data))

# WRANGLING FUNCS

In [11]:
answer = raw_data['answer']
answer['survey'] = answer['survey'].apply(lambda x: re.split(' (?=@)',x)[0])
answer['user'] = answer['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [12]:
answer = answer.pivot_table(index='user', 
                            columns='survey', 
                            values='answer', 
                            aggfunc=lambda x: ' '.join(str(v) for v in x))
answer = answer.add_suffix('_surv')

In [13]:
#user = raw_data['user'][['_id',
 #                        'active',
  #                       'dob',
   #                      'gender',
    #                     'heightFeet',
     #                    'heightInches',
      #                   'joined',
       #                  'weight']]

In [14]:
#main = user.merge(answer, 
 #                 how='left', 
  #                left_on='_id', 
   #               right_index=True, 
    #              sort=True)

In [15]:
main = raw_data['user'][['_id',
                         'active',
                         'dob',
                         'gender',
                         'heightFeet',
                         'heightInches',
                         'joined',
                         'weight']]

In [16]:
main['dob'] = main['dob'].apply(lambda x: pd.to_datetime(x))
main['joined'] = main['joined'].apply(lambda x: pd.to_datetime(x))

main['age'] = main['dob'].apply(lambda x: relativedelta.relativedelta(date.today(), datetime.date(x)).years)
main['act_time'] = main['joined'].apply(lambda x: (date.today() - datetime.date(x)).days)

In [17]:
chal_inv = raw_data['challengeInvitation']
chal_inv['challenge'] = chal_inv['challenge'].apply(lambda x: re.split(' (?=@)',x)[0])
chal_inv['friend'] = chal_inv['friend'].apply(lambda x: re.split(' (?=@)',x)[0])
chal_inv['user'] = chal_inv['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [18]:
chal_inv = chal_inv.pivot_table(index='user',
                                columns='challenge', 
                                values='friend',
                                aggfunc=lambda x: ' '.join(str(v) for v in x))
chal_inv = chal_inv.add_suffix('_chalinv')

In [19]:
#main = main.merge(chal_inv,
 #                 how='left',
  #                left_on='_id',
   #               right_index=True,
    #              sort=True)

Don't understand relationships between "challenger" table and "challengeInvitation" table 'user' column

In [20]:
frnd_rq = raw_data['friendship']
frnd_rq['friend'] = frnd_rq['friend'].apply(lambda x: re.split(' (?=@)',x)[0])
frnd_rq['user'] = frnd_rq['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [21]:
frnd_rq = frnd_rq.pivot_table(index='user',
                              columns='friend',
                              values='status',
                              aggfunc=lambda x: ' '.join(str(v) for v in x))
frnd_rq = frnd_rq.add_suffix('_friendid')

In [22]:
#main = main.merge(frnd_rq,
 #                 how='left',
  #                left_on='_id',
   #               right_index=True,
    #              sort=True)

In [23]:
event_ast = raw_data['assistantEvent']
event_ast['event'] = event_ast['event'].apply(lambda x: re.split(' (?=@)',x)[0])
event_ast['user'] = event_ast['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [24]:
event_ast = event_ast.pivot_table(index='user',
                                  columns='event',
                                  values='_id',
                                  aggfunc=lambda x: ' '.join(str(v) for v in x))
event_ast = event_ast.add_suffix('_eventid')

In [25]:
#main = main.merge(event_ast,
 #                 how='left',
  #                left_on='_id',
   #               right_index=True,
    #              sort=True)

In [26]:
post = raw_data['post']
post['user'] = post['user'].apply(lambda x: re.split(' (?=@)',x)[0])
post['item._class'] = post['item._class'].apply(lambda x: re.sub('[a-z].+\.(?=[A-Z])', '', str(x)))

In [27]:
post['counter'] = int(1)
post = post.pivot_table(index='user',
                        columns='item._class',
                        values='counter',
                        aggfunc='sum')

In [28]:
main = main.merge(post,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

In [29]:
metrics_miles_time = raw_data['post'][['user','item.miles','item.time','item.positions']]
metrics_miles_time = metrics_miles_time.groupby(metrics_miles_time.user).sum()

metrics_exercise = raw_data['post'][['user','item.exercise']]
metrics_exercise['counter'] = int(1)

metrics_chals_sent = pd.DataFrame(chal_inv.count(axis=1), columns={'chal_sent'})
metrics_chals_unique = pd.DataFrame(chal_inv.nunique(axis=1), columns={'chal_unique'})
metrics_chals_uratio = metrics_chals_sent.join(metrics_chals_unique)
metrics_chals_uratio['chal_uni_frnd'] = metrics_chals_uratio['chal_sent']/metrics_chals_uratio['chal_unique']

metrics_frnd_sent = pd.DataFrame(frnd_rq.count(axis=1), columns={'frnd_sent'})

In [30]:
metrics_exercise = metrics_exercise.pivot_table(index='user',
                                                columns='item.exercise',
                                                values='counter',
                                                aggfunc='sum')

In [31]:
main = main.merge(metrics_miles_time,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

main = main.merge(metrics_exercise,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

main = main.merge(metrics_chals_uratio,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

main = main.merge(frnd_rq.apply(pd.value_counts, axis=1)[['ACCEPTED','PENDING','WAITING']],
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

main.drop(353, inplace=True) #Record is all NaN
main.drop(['dob','joined','CompanyMessage'], axis=1, inplace=True)

In [32]:
#event = main.filter(regex='_eventid')
#friend = main.filter(regex='_friendid')

GIVE 'EVENT' AND 'FRIEND' THEIR OWN TABLES AND SCORE THEM ADDED TO 'MAIN' DATA FRAME

EXTRA COLUMNS LIKE "ACCEPTED ACCEPTED" HAVE A 1 OCCASIONALLY. NOT SURE IF ISSUE WITH RAW DATA OR 'APPLY' FUNCTION

In [33]:
split = np.split(main, [8], axis=1)
nas = split[1].isnull()
split[1][nas] = 0

In [35]:
split

[                          _id active  gender heightFeet heightInches  weight  \
 185  54f530089932b01dbcffc89b   True  FEMALE          5            3  143.75   
 134  54f530c89932b01dbcffc89c   True    MALE          5           11     170   
 110  54f5ef739932b01dbcffc8a2   True    MALE          6          NaN   194.9   
 198  54f757f4993241fc498ec5d7   True    MALE          5           11     191   
 228  54f8f6f59932ce1b71e7317a   True     NaN        NaN          NaN     NaN   
 43   54fe33869932ce1b71e73193   True    MALE          5           10     180   
 53   54fe49989932ce1b71e73194   True    MALE        NaN          NaN     NaN   
 0    5509a55b993212d9d6de6e35   True     NaN        NaN          NaN     NaN   
 315  550a054e993212d9d6de6e3e   True    MALE          5           11   185.8   
 15   550a1875993212d9d6de6e4e   True    MALE          5           10     185   
 446  550a6c30993212d9d6de6e63   True  FEMALE          5            2     135   
 21   550af727993212d9d6de6e

In [34]:
main = split[0].join(split[1])
main = main.join(pd.get_dummies(main['gender']))

In [8]:
clust_set = main.loc[:,'heightFeet':]
clust_set.fillna(clust_set.median(), inplace=True)

,heightFeet,heightInches,joined,weight,FEMALE,MALE
0,5.0,6.0,2015-03-18T16:18:35.099Z,160.0,0,0
1,5.0,7.0,2015-03-19T18:52:00.922Z,110.0,1,0
2,5.0,3.0,2015-03-19T19:02:20.091Z,160.0,1,0
3,5.0,11.0,2015-03-20T22:30:22.928Z,200.0,0,1
4,5.0,6.0,2015-03-19T20:26:02.633Z,160.0,0,0
5,5.0,2.0,2015-03-21T01:58:15.721Z,125.0,0,1
6,6.0,0.0,2015-03-23T17:52:54.643Z,180.0,0,1
7,5.0,6.0,2015-03-30T18:20:47.819Z,160.0,1,0
8,5.0,11.0,2015-04-06T21:05:59.535Z,165.0,0,0
9,5.0,6.0,2015-04-06T21:08:21.392Z,160.0,1,0


In [41]:
clust_set.describe()
list(clust_set)

['heightFeet',
 'heightInches',
 'weight',
 'age',
 'act_time',
 'BloodPressureCheckIn',
 'Challenger',
 'CholesterolCheckIn',
 'EventCheckIn',
 'FitbitCheckIn',
 'FoodCheckIn',
 'GlucoseCheckIn',
 'GymCheckIn',
 'MovesChallenge',
 'MovesCheckIn',
 'PictureChallenge',
 'RunningCheckIn',
 'WeightCheckIn',
 'YogaCheckIn',
 'item.miles',
 'item.time',
 'item.positions',
 'CARDIO',
 'FITNESS',
 'OTHER',
 'SWIMMING',
 'WEIGHTS',
 'chal_sent',
 'chal_unique',
 'chal_uni_frnd',
 'ACCEPTED',
 'PENDING',
 'WAITING',
 'FEMALE',
 'MALE',
 'AC_label']

In [30]:
ss = preprocessing.StandardScaler(with_mean=False)
mms = preprocessing.MinMaxScaler()

In [31]:
processed = pd.DataFrame(ss.fit_transform(clust_set), columns=clust_set.columns, index=clust_set.index)
mms_processed = pd.DataFrame(mms.fit_transform(clust_set), columns=clust_set.columns, index=clust_set.index)

In [42]:
processed.describe()
mms_processed.describe()

,heightFeet,heightInches,weight,age,act_time,BloodPressureCheckIn,Challenger,CholesterolCheckIn,EventCheckIn,FitbitCheckIn,...,SWIMMING,WEIGHTS,chal_sent,chal_unique,chal_uni_frnd,ACCEPTED,PENDING,WAITING,FEMALE,MALE
count,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,...,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000
mean,0.043970,0.088858,0.025789,0.448426,0.598670,0.011356,0.058083,0.005085,0.027986,0.010302,...,0.004242,0.019184,0.055932,0.068362,0.055650,0.033975,0.093785,0.011723,0.400000,0.555932
std,0.043246,0.061735,0.040536,0.137027,0.313436,0.051693,0.100873,0.059652,0.097229,0.062288,...,0.047437,0.089710,0.148988,0.173833,0.143493,0.084512,0.143370,0.068161,0.490314,0.497283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.038462,0.058824,0.021488,0.400000,0.319121,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.038462,0.088235,0.023604,0.442857,0.609110,0.000000,0.038462,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004505,0.027778,0.000000,0.000000,1.000000
75%,0.038462,0.117647,0.026046,0.500000,0.939619,0.000000,0.076923,0.000000,0.005814,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.166667,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
processed

,heightFeet,heightInches,weight,age,act_time,BloodPressureCheckIn,Challenger,CholesterolCheckIn,EventCheckIn,FitbitCheckIn,...,SWIMMING,WEIGHTS,chal_sent,chal_unique,chal_uni_frnd,ACCEPTED,PENDING,WAITING,FEMALE,MALE
185,2.225310,0.715237,0.577769,3.652020,3.308162,5.808401,4.197726,11.185462,0.119695,0.000000,...,0.000000,0.000000,1.343531,1.919179,1.743721,0.053345,0.581741,0.000000,2.041241,0.000000
134,2.225310,2.622535,0.683275,4.695454,3.308162,0.000000,5.724171,11.185462,0.059847,0.000000,...,0.000000,0.000000,2.687063,1.919179,3.487441,0.960216,1.357395,0.000000,0.000000,2.012632
110,2.670372,1.430474,0.783355,0.208687,3.308162,0.000000,3.052891,5.592731,0.000000,0.626033,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.426763,0.000000,0.050985,0.000000,2.012632
198,2.225310,2.622535,0.767680,3.443333,3.308162,19.361337,4.579337,5.592731,0.000000,0.000000,...,0.000000,0.027752,4.030594,3.838358,2.615581,0.320072,0.000000,0.050985,0.000000,2.012632
228,2.225310,1.430474,0.643082,3.547676,3.308162,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.053345,0.000000,0.000000,0.000000,0.000000
43,2.225310,2.384123,0.723468,4.382424,3.308162,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.160036,0.000000,0.000000,0.000000,2.012632
53,2.225310,1.430474,0.643082,0.626061,3.308162,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.053345,0.193914,0.000000,0.000000,2.012632
0,2.225310,1.430474,0.643082,3.234646,3.308162,0.000000,1.908057,0.000000,0.059847,0.000000,...,0.000000,0.000000,5.374126,1.919179,6.974882,1.600360,1.939135,0.152954,0.000000,0.000000
315,2.225310,2.622535,0.746779,2.817272,3.308162,1.936134,4.960948,0.000000,0.299237,16.068182,...,0.000000,0.000000,4.030594,3.838358,2.615581,2.507231,0.193914,0.000000,0.000000,2.012632
15,2.225310,2.384123,0.743564,3.234646,3.304779,1.936134,0.381611,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.041751,0.000000,0.000000,2.012632


In [34]:
from sklearn.decomposition import PCA, SparsePCA, FactorAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [35]:
fa = FeatureAgglomeration(n_clusters=5, affinity='cosine', linkage='complete')
feats = fa.fit_transform(processed)
test = pd.DataFrame(feats, index=processed.index)
agg_model = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='complete')
model = agg_model.fit(test).labels_

In [36]:
clust_set['AC_label'] = model

In [37]:
clust_set.groupby('AC_label').describe().T

AC_label                               0             1            2
ACCEPTED             count    576.000000      7.000000     7.000000
                     mean       6.725694     76.857143     5.428571
                     std       15.774314     70.664533     6.373307
                     min        0.000000     20.000000     0.000000
                     25%        0.000000     36.500000     0.500000
                     50%        1.000000     47.000000     5.000000
                     75%        6.000000     88.000000     7.000000
                     max      152.000000    222.000000    18.000000
BloodPressureCheckIn count    576.000000      7.000000     7.000000
                     mean       0.088542      0.428571     1.857143
                     std        0.290380      0.534522     3.760699
                     min        0.000000      0.000000     0.000000
                     25%        0.000000      0.000000     0.000000
                     50%        0.000000      0.000000     0.000000
                     75%        0.000000      1.000000     1.500000
                     max        2.000000      1.000000    10.000000
CARDIO               count    576.000000      7.000000     7.000000
                     mean       4.362847     15.000000     1.714286
                     std       23.488431     32.726136     2.360387
                     min        0.000000      0.000000     0.000000
                     25%        0.000000      0.000000     0.000000
                     50%        0.000000      0.000000     0.000000
                     75%        0.000000      8.500000     3.000000
                     max      337.000000     88.000000     6.000000
Challenger           count    576.000000      7.000000     7.000000
                     mean       1.324653     11.571429     6.714286
                     std        2.084431      8.121107     6.317022
                     min        0.000000      2.000000     0.000000
                     25%        0.000000      6.000000     0.500000
                     50%        1.000000     13.000000     8.000000
...                                  ...           ...          ...
item.miles           std      121.370598     97.457045    28.709282
                     min        0.000000      0.000000     0.000000
                     25%        0.000000      0.000000    16.350000
                     50%        0.000000     63.250000    33.500000
                     75%        3.000000    135.430000    41.250000
                     max     1025.000000    255.500000    87.500000
item.positions       count    576.000000      7.000000     7.000000
                     mean      19.930556    276.571429     3.571429
                     std      111.015543    421.553815     4.649629
                     min        0.000000      0.000000     0.000000
                     25%        0.000000      2.500000     0.500000
                     50%        0.000000      6.000000     2.000000
                     75%        0.000000    410.000000     4.500000
                     max     1290.000000   1105.000000    13.000000
item.time            count    576.000000      7.000000     7.000000
                     mean    2309.098958   7505.857143   524.142857
                     std     5696.972029  10444.897980   284.825928
                     min        0.000000      5.000000   186.000000
                     25%        0.000000    520.000000   326.500000
                     50%       55.000000   1633.000000   555.000000
                     75%     1563.750000  10694.000000   622.500000
                     max    45140.000000  28475.000000  1030.000000
weight               count    576.000000      7.000000     7.000000
                     mean     173.663021    143.685714   183.065000
                     std      251.950935     28.811307    40.645158
                     min       15.000000    102.000000   143.750000
                     25%      147.750000    126.500000  

In [38]:
lda = LinearDiscriminantAnalysis()
clust_lda = lda.fit_transform(clust_set, clust_set['AC_label'])

In [39]:
pd.DataFrame(lda.coef_, columns=clust_set.columns).describe()

,heightFeet,heightInches,weight,age,act_time,BloodPressureCheckIn,Challenger,CholesterolCheckIn,EventCheckIn,FitbitCheckIn,...,WEIGHTS,chal_sent,chal_unique,chal_uni_frnd,ACCEPTED,PENDING,WAITING,FEMALE,MALE,AC_label
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0
mean,-0.096924,0.327792,0.000235,-0.063629,-0.000534,-18.017789,-0.419422,26.416113,0.092953,0.947112,...,-0.253809,-6.042737,7.046046,1.843730,-0.010611,0.108462,0.023851,2.697567,4.164390,0.0
std,0.178716,0.478974,0.000801,0.129969,0.002677,27.996363,1.380976,58.054271,0.083713,1.942380,...,0.228159,6.779654,11.834308,1.655664,0.428099,0.231463,0.160341,5.288631,6.475892,0.0
min,-0.303264,-0.012098,-0.000416,-0.213353,-0.003444,-50.206755,-1.965496,-12.875118,-0.003431,-0.308136,...,-0.395891,-13.239965,-0.260047,-0.068046,-0.444106,-0.045278,-0.122688,-0.605114,-0.153695,0.0
25%,-0.149844,0.053893,-0.000212,-0.105502,-0.001712,-27.359174,-0.975008,-6.925027,0.065678,-0.171546,...,-0.385397,-9.175614,0.219108,1.362027,-0.221857,-0.024641,-0.061784,-0.352336,0.441349,0.0
50%,0.003577,0.119884,-0.000009,0.002348,0.000020,-4.511593,0.015480,-0.974936,0.134787,-0.034955,...,-0.374903,-5.111263,0.698263,2.792100,0.000392,-0.004003,-0.000880,-0.099559,1.036392,0.0
75%,0.006246,0.497736,0.000561,0.011233,0.000921,-1.923306,0.353615,46.061729,0.141145,1.574736,...,-0.182768,-2.444122,10.699093,2.799618,0.206136,0.185333,0.097120,4.348907,6.323432,0.0
max,0.008915,0.875589,0.001130,0.020117,0.001823,0.664980,0.691750,93.098393,0.147503,3.184426,...,0.009367,0.223018,20.699923,2.807135,0.411881,0.374668,0.195121,8.797373,11.610472,0.0


## DRAFT SCRIPTS

In [40]:
agg = cluster.AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage='complete')
labels = agg.fit_predict(processed)
clust_set['Agg_Clust'] = labels
clust_set['Agg_Clust'].value_counts()

NameError: name 'cluster' is not defined

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Agg_Clust', y='heightInches', data=clust_set)

In [ ]:
sns.pairplot(data=processed.iloc[:,5:20])

In [ ]:
dbs = cluster.DBSCAN(eps=2, min_samples=6)
labels = dbs.fit_predict(processed)
clust_set['DBS_Clust'] = labels
clust_set['DBS_Clust'].value_counts()

In [ ]:
clust_set

In [ ]:
sc = cluster.SpectralClustering(n_clusters=4)
labels = sc.fit_predict(processed)
clust_set['SC_Clust'] = labels
clust_set['SC_Clust'].value_counts()

In [ ]:
km = cluster.KMeans(n_clusters=4)
labels = km.fit_predict(processed)
clust_set['KM_Clust'] = labels
clust_set['KM_Clust'].value_counts()

In [ ]:
clust_set.columns

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Agg_Clust', y='chal_uni_frnd', data=clust_set)

In [ ]:
metrics.silhouette_score(clust_set, labels=clust_set['KM_Clust'], metric='cosine')

In [ ]:
metrics.calinski_harabaz_score(clust_set, labels=clust_set['KM_Clust'])

In [ ]:
metrics.calinski_harabaz_score(clust_set, labels=clust_set['KM_Clust'])